As an aside, there is no ISO standard for quarters, but there is an EDTF recommended by the ISO format (see: https://en.wikipedia.org/wiki/ISO_8601#Standardised_extensions and https://www.loc.gov/standards/datetime/). Unfortunately, the datetime formats of Python don't recognize EDTF formats. Instead I'll be replacing the quarter designations with the month number for the first month in the quarter (e.g. 1 - January, 4 - April, etc)

In [29]:
import pandas as pd

bee_colony_data = pd.read_csv('bees_post_eda.csv')
bee_colony_data.head(5)

,State,year,quarter,Starting colonies,Maximum colonies,Lost colonies,Percent lost,Added colonies,Varroa mites,Other pests,Diseases,Pesticides,Other,Unknown,year_and_quarter,region,subregion
0,Alabama,2015,Q1,7000.0,7000.0,1800.0,26.0,2800.0,10.0,5.4,NaN,2.2,9.1,9.4,2015 Q1,south,east south central
1,Alabama,2015,Q2,7500.0,7500.0,860.0,12.0,1900.0,16.7,42.5,NaN,2.3,3.2,4.1,2015 Q2,south,east south central
2,Alabama,2015,Q3,8500.0,9000.0,1400.0,16.0,160.0,63.1,70.6,NaN,2.6,2.2,17.7,2015 Q3,south,east south central
3,Alabama,2015,Q4,8000.0,8000.0,610.0,8.0,80.0,3.1,6.4,0.2,0.2,2.8,1.9,2015 Q4,south,east south central
4,Alabama,2016,Q1,7500.0,7500.0,1700.0,23.0,1200.0,24.2,22.0,4.3,8.1,2.4,11.3,2016 Q1,south,east south central


Convert year and quarter to EDTF format

In [30]:
bee_colony_data['year_and_quarter'] = bee_colony_data['year_and_quarter'].str.replace(' Q1', '-01')
bee_colony_data['year_and_quarter'] = bee_colony_data['year_and_quarter'].str.replace(' Q2', '-04')
bee_colony_data['year_and_quarter'] = bee_colony_data['year_and_quarter'].str.replace(' Q3', '-07')
bee_colony_data['year_and_quarter'] = bee_colony_data['year_and_quarter'].str.replace(' Q4', '-10')

bee_colony_data['year_and_quarter'] = pd.to_datetime(bee_colony_data['year_and_quarter'], format='%Y-%m')

State                        object
year                          int64
quarter                      object
Starting colonies           float64
Maximum colonies            float64
Lost colonies               float64
Percent lost                float64
Added colonies              float64
Varroa mites                float64
Other pests                 float64
Diseases                    float64
Pesticides                  float64
Other                       float64
Unknown                     float64
year_and_quarter     datetime64[ns]
region                       object
subregion                    object
dtype: object

Create dummy variables for states, regions, and subregions

In [33]:
bee_data_encoded = pd.get_dummies(bee_colony_data, columns=['State', 'region', 'subregion'])

Apply StandardScaler to the numeric columns after separating the dependent and target variables

In [34]:
from sklearn.preprocessing import StandardScaler

scaled_col_names = ['Percent lost', 'Added colonies', 'Varroa mites', 'Other pests', 'Diseases', 'Pesticides', 
                    'Other', 'Unknown']

scaled_features = bee_data_encoded[scaled_col_names]
scaler = StandardScaler()
scaled_bee_data = scaler.fit_transform(scaled_features.values)

bee_data_encoded[scaled_col_names] = scaled_bee_data
bee_data_encoded.head(5)

,year,quarter,Starting colonies,Maximum colonies,Lost colonies,Percent lost,Added colonies,Varroa mites,Other pests,Diseases,...,region_west,subregion_east north central,subregion_east south central,subregion_mid atlantic,subregion_mountain,subregion_new england,subregion_pacific,subregion_south atlantic,subregion_west north central,subregion_west south central
0,2015,Q1,7000.0,7000.0,1800.0,1.905818,-0.269072,-0.998010,-0.438503,NaN,...,0,0,1,0,0,0,0,0,0,0
1,2015,Q2,7500.0,7500.0,860.0,0.085000,-0.301820,-0.649452,2.348027,NaN,...,0,0,1,0,0,0,0,0,0,0
2,2015,Q3,8500.0,9000.0,1400.0,0.605234,-0.365133,1.764442,4.458580,NaN,...,0,0,1,0,0,0,0,0,0,0
3,2015,Q4,8000.0,8000.0,610.0,-0.435234,-0.368044,-1.356972,-0.363395,-0.528829,...,0,0,1,0,0,0,0,0,0,0
4,2016,Q1,7500.0,7500.0,1700.0,1.515643,-0.327291,-0.259275,0.808300,-0.020045,...,0,0,1,0,0,0,0,0,0,0


Create a train and test split

In [58]:
from sklearn.model_selection import train_test_split

# Drop unused columns before splitting
bee_data_encoded.drop(['year', 'quarter'], axis=1, inplace=True)
X = bee_data_encoded.drop('Percent lost', axis=1)
y = bee_data_encoded['Percent lost']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [59]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
bee_data_encoded.to_csv('bee_data_after_preprocessing.csv', index=False)